In [2]:
import os
import sys, getopt
import time, datetime
from pydub import AudioSegment
import pydub
import subprocess
import pandas as pd
import numpy as np
import math
from copy import copy
from random import random

os.chdir(os.path.expanduser('~/Dropbox/aapb-hipstas'))

In [3]:
import glob
import fnmatch
import os

matches = []
audio_paths=["/Volumes/Turcich-2012/AAPB_Test_Haystack_16000",\
"/Users/mclaugh/Desktop/AAPB_400_hrs_labeled/wav_16000/",\
"/Volumes/McLaughlin-6TB-1/Extended_Corpus/"]



matches_mp4 = []

matches_mp3 = []

matches_wav = []



for dir_path in audio_paths:
    for root, dirnames, filenames in os.walk(dir_path):
        for filename in fnmatch.filter(filenames, '*.mp4'):
            matches_mp4.append(os.path.join(root, filename))
        for filename in fnmatch.filter(filenames, '*.mp3'):
            matches_mp3.append(os.path.join(root, filename))
        for filename in fnmatch.filter(filenames, '*.16000.wav'):
            matches_wav.append(os.path.join(root, filename))

#matches=list(set(matches_mp4+matches_mp3+matches_wav))

matches=matches_wav

len(matches)

2153

In [4]:
basename_to_path={}

for pathname in matches:
    basename=pathname.split('/')[-1][:-4]  # removing file extension and rest of path
    if basename[-6:]=='.16000':
        basename = basename[:-6]
    basename_to_path[basename]=pathname

print(basename_to_path['cpb-aacip-15-579s51qt.h264'])

/Volumes/McLaughlin-6TB-1/Extended_Corpus/Malcolm_X/cpb-aacip-15-579s51qt.h264.16000.wav


In [5]:
print(basename_to_path['cpb-aacip-15-9995xnw4'])

/Users/mclaugh/Desktop/AAPB_400_hrs_labeled/wav_16000/cpb-aacip-15-9995xnw4.16000.wav


In [6]:
aapb_metadata_all=pd.read_csv("AAPB_ARLO_All_170412.csv")
aapb_metadata_all.tail(2)

,AAPB Unique Identifier (GUID),Type,Value,class ID,Timecode IN,Timecode OUT,Tag Duration,File Duration,SonyCi ID,Filename
2984,NaN,NaN,"Nixon, Richard",Richard Nixon,2378.884354,2389.031474,10.147120,3544.062,NaN,cpb-aacip-75-88qbzvhx.h264
2985,NaN,NaN,"Nixon, Richard",Richard Nixon,2433.962086,2439.674195,5.712109,3544.062,NaN,cpb-aacip-75-88qbzvhx.h264


In [7]:
#Adding current pathname for each row

aapb_metadata_all['Pathname']=''

for i in range(len(aapb_metadata_all)):
    row=aapb_metadata_all.iloc[i]
    basename=row['Filename']
    try:
        #print(str(dur_dict[basename]))
        aapb_metadata_all.set_value(i, 'Pathname', basename_to_path[basename])
    except:
        print(basename)
    


nan
nan
nan
nan
nan
cpb-aacip-15-6688g8fn75__25311-2010_09_15_.h264
cpb-aacip-15-6688g8fn75__25311-2010_09_15_.h264


In [8]:
aapb_metadata_all.head(3)

,AAPB Unique Identifier (GUID),Type,Value,class ID,Timecode IN,Timecode OUT,Tag Duration,File Duration,SonyCi ID,Filename,Pathname
0,cpb-aacip/500-j9609t48,Significant speaker voice,"Kennedy, John F.",NaN,5.0,110.0,105.0,1739.184,6d54987efa73413cb3558c95baa907a6,cpb-aacip-500-j9609t48,/Volumes/McLaughlin-6TB-1/Extended_Corpus/John...
1,cpb-aacip/500-j9609t48,Sound quality,audience clapping,NaN,131.0,140.0,9.0,1739.184,6d54987efa73413cb3558c95baa907a6,cpb-aacip-500-j9609t48,/Volumes/McLaughlin-6TB-1/Extended_Corpus/John...
2,cpb-aacip/500-j9609t48,Significant speaker voice,"Kennedy, John F.",NaN,119.0,254.0,135.0,1739.184,6d54987efa73413cb3558c95baa907a6,cpb-aacip-500-j9609t48,/Volumes/McLaughlin-6TB-1/Extended_Corpus/John...


In [9]:
# finding all "Value" labels
label_values=sorted([str(item) for item in list(set(list(aapb_metadata_all['Value'])))])
label_values.remove('nan')
print(label_values)

['Baldwin, James', 'Buckley, William F.', 'Bush, George HW', 'Bush, George W.', 'Carter, Jimmy', 'Child, Julia', 'Classical Music', 'Clinton, Bill', 'Clinton, Hillary', 'Dog Bark', 'Ford, Gerald', 'Galbraith, John Kenneth', 'Gore, Al', 'Johnson, Lyndon', 'Kennedy, John F.', 'King, Martin Luther', 'Kissinger, Henry', 'Lehrer, Jim', 'NPR embedded feed', 'Nixon, Richard', 'PRI embedded feed', 'Pepin, Jacques', 'Reagan, Ronald', 'Spanish Language', 'Steinem, Gloria', 'Vidal, Gore', 'X, Malcolm', 'applause', 'audience clapping']


In [10]:
aapb_metadata_all_split=pd.DataFrame(columns=['AAPB Unique Identifier (GUID)', 'Type', 'Value', 'class ID',\
       'Timecode IN', 'Timecode OUT', 'Tag Duration', 'File Duration',\
       'SonyCi ID', 'Filename', 'Pathname'])



for value in label_values:

    x_table=aapb_metadata_all[aapb_metadata_all['Value']==value]

    for index, rowref in x_table.iterrows():
        row=aapb_metadata_all.iloc[index]
        num_clips=math.trunc(float(row['Tag Duration'])/2) #checking how many solid 2-second clips we can extract
        #print(num_clips)
        #break
        row_list=list(aapb_metadata_all.iloc[index])
        series_start=row["Timecode IN"]
        list_of_lists=[]
    
        for i in range(num_clips):
            row_list=list(aapb_metadata_all.iloc[index])
            row_list[4]=float(row_list[4])+(2.0*i)
            row_list[5]=float(row_list[4])+2.0
            row_list[6]=2.0
            list_of_lists.append(copy(row_list))
            #print(row_list)
        
        table=pd.DataFrame(list_of_lists,columns=['AAPB Unique Identifier (GUID)', 'Type', 'Value', 'class ID',\
               'Timecode IN', 'Timecode OUT', 'Tag Duration', 'File Duration',\
               'SonyCi ID', 'Filename', 'Pathname'])
    
        aapb_metadata_all_split=aapb_metadata_all_split.append(table,ignore_index=True)

In [11]:

print(np.sum(list(aapb_metadata_all[aapb_metadata_all['Value']=="King, Martin Luther"]['Tag Duration'])))

print(np.sum(list(aapb_metadata_all_split[aapb_metadata_all_split['Value']=="King, Martin Luther"]['Tag Duration'])))


42407.0
42358.0


In [12]:
print(len(list(aapb_metadata_all[aapb_metadata_all['Value']=="King, Martin Luther"]['Filename'])))
print(len(list(aapb_metadata_all_split[aapb_metadata_all_split['Value']=="King, Martin Luther"]['Filename'])))

95
21179


In [13]:
aapb_metadata_all_split.head(3)

,AAPB Unique Identifier (GUID),Type,Value,class ID,Timecode IN,Timecode OUT,Tag Duration,File Duration,SonyCi ID,Filename,Pathname
0,cpb-aacip/28-td9n29pp9j,Significant speaker voice,"Baldwin, James",NaN,32.0,34.0,2.0,1605.408,f965cf9612054347aabe2cbdf87cd487,cpb-aacip-28-td9n29pp9j__PRA_AAPP_BC0642_James...,/Volumes/McLaughlin-6TB-1/Extended_Corpus/Mart...
1,cpb-aacip/28-td9n29pp9j,Significant speaker voice,"Baldwin, James",NaN,34.0,36.0,2.0,1605.408,f965cf9612054347aabe2cbdf87cd487,cpb-aacip-28-td9n29pp9j__PRA_AAPP_BC0642_James...,/Volumes/McLaughlin-6TB-1/Extended_Corpus/Mart...
2,cpb-aacip/28-td9n29pp9j,Significant speaker voice,"Baldwin, James",NaN,36.0,38.0,2.0,1605.408,f965cf9612054347aabe2cbdf87cd487,cpb-aacip-28-td9n29pp9j__PRA_AAPP_BC0642_James...,/Volumes/McLaughlin-6TB-1/Extended_Corpus/Mart...


In [17]:
aapb_metadata_all_split.sample(n=3)

,AAPB Unique Identifier (GUID),Type,Value,class ID,Timecode IN,Timecode OUT,Tag Duration,File Duration,SonyCi ID,Filename,Pathname
127421,cpb-aacip/15-04dncrzg,Significant speaker voice,"Reagan, Ronald",NaN,188.00,190.00,2.0,2777.760,ed6bff6cce104b79a41bf1097cb42423,cpb-aacip-15-04dncrzg,/Volumes/McLaughlin-6TB-1/Extended_Corpus/Rona...
3565,cpb-aacip/15-687h4jgd,Significant speaker voice,"Baldwin, James",NaN,843.00,845.00,2.0,3588.120,995bcf24bb85481694753c567d021f58,cpb-aacip-15-687h4jgd,/Volumes/McLaughlin-6TB-1/Extended_Corpus/Jame...
110203,NaN,NaN,"Nixon, Richard",RIchard Nixon,1289.95,1291.95,2.0,1790.592,NaN,cpb-aacip-15-569329x3,/Volumes/McLaughlin-6TB-1/Extended_Corpus/Rich...


In [18]:
##writing csv
aapb_metadata_all_split.to_csv('AAPB_ARLO_All_170414_2_sec_segs.csv',index=False)

In [ ]:
!pwd

In [ ]:
row=aapb_metadata_all_split.iloc[9]
row["Pathname"]

In [ ]:
os.chdir('/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/Model_training_clips')

In [ ]:

def extract_clips(x_table):
    for unique_audio_path in sorted(list(set(list(x_table['Pathname'])))):
        inputfile=unique_audio_path
        if os.path.exists(inputfile):
            if inputfile.lower()[-4:].lower() in ('.wav','.mp3','.mp4'):
                wav_source=True
                if inputfile.lower()[-4:]=='.mp4':     # Creates a temporary WAV
                    wav_source=False                         # if input is MP4
                    temp_filename=inputfile.split('/')[-1]+'_temp.wav'
                    audio_path='/var/tmp/'+temp_filename   # Pathname for temp WAV
                    subprocess.call(['ffmpeg', '-y', '-i', inputfile, audio_path]) # '-y' option overwrites existing file if present
                else:
                    audio_path=inputfile

                song=None
                from pydub import AudioSegment
                
                if inputfile[-4:].lower()=='.mp3':
                    song = AudioSegment.from_mp3(audio_path)
                else:
                    song = AudioSegment.from_wav(audio_path)


        for index, rowref in x_table[x_table['Pathname']==unique_audio_path].iterrows():
            row=aapb_metadata_all_split.iloc[index] ## Ref to master split table
            time_in=row['Timecode IN']
            clip_duration=row['Tag Duration']
            inputfile=row['Pathname']
            basename=row['Filename']
            dir_name=row['Value']
            
            try: os.mkdir(dir_name)
            except: pass

        
            from pydub import AudioSegment 
            start_msec = float(time_in) * 1000.0
            duration_msec = float(clip_duration) * 1000
            clip_pathname=basename+'.start_'+str(time_in)[:8]+'.dur_2s.wav'
            if not os.path.exists(os.path.join(dir_name,clip_pathname)):
                clip_data = song[start_msec:start_msec+duration_msec]
                clip_data=clip_data.set_channels(1)
                clip_data.export(os.path.join(dir_name,clip_pathname), format="wav")

        try:
            if wav_source==False:
                os.remove(audio_path)
        except: pass

    print("*** All segments extracted! ***")



In [ ]:
##### Running the Random Clip Extractor #####

name="Baldwin, James"
num_clips=846     ## number of 2-second clips desired
print(len(aapb_metadata_all_split[aapb_metadata_all_split['Value']==name]))


x_table=aapb_metadata_all_split[aapb_metadata_all_split['Value']==name].sample(n=num_clips)

extract_clips(x_table)



In [ ]:
#extract_clips(x_table)

In [ ]:
# Get files not included

subset_paths=set(list(x_table["Pathname"]))
all_paths=set(list(aapb_metadata_all_split[aapb_metadata_all_split['Value']==name]["Pathname"]))
list(all_paths-subset_paths)

In [ ]:
## Loading table of labeled 2-second audio segments

aapb_metadata_all_split_1=pd.read_csv("/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/AAPB_ARLO_All_170414_2_sec_segs.csv")
aapb_metadata_all_split_1.tail(1)

len(list(aapb_metadata_all_split_1[aapb_metadata_all_split_1['Value']=="King, Martin Luther"]['Filename']))

In [ ]:

## Loading table of labeled 2-second audio segments

aapb_metadata_all_0=pd.read_csv("/Volumes/McLaughlin-6TB-1/Dropbox/aapb-hipstas/AAPB_ARLO_All_170412.csv")
aapb_metadata_all_0.tail(1)

len(list(aapb_metadata_all_0[aapb_metadata_all_0['Value']=="King, Martin Luther"]['Filename']))

In [ ]:


len(list(aapb_metadata_all_split[aapb_metadata_all_split['Value']=="King, Martin Luther"]['Filename']))



np.sum(list(aapb_metadata_all_split[aapb_metadata_all_split['Value']=="King, Martin Luther"]['Tag Duration']))

In [ ]:


len(list(aapb_metadata_all[aapb_metadata_all['Value']=="King, Martin Luther"]['Filename']))



np.sum(list(aapb_metadata_all[aapb_metadata_all['Value']=="King, Martin Luther"]['Tag Duration']))